<a href="https://colab.research.google.com/github/atharva-729/Movie-Recommendation-System/blob/main/Movie_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opendatasets

In [ ]:
import opendatasets as od
import pandas as pd

od.download("https://www.kaggle.com/datasets/tmdb/tmdb-movie-metadata/")

In [ ]:
import matplotlib.pyplot as plt
import json

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

In [ ]:
%cd /content/tmdb-movie-metadata

In [ ]:
movies = pd.read_csv("tmdb_5000_movies.csv")

In [ ]:
movies.head()

In [ ]:
credits = pd.read_csv("tmdb_5000_credits.csv")

In [ ]:
credits.head()

In [ ]:
credits.shape

In [ ]:
movies.shape

In [ ]:
movies.isna().sum()

In [ ]:
movies.columns

In [ ]:
movies.id

In [ ]:
movies.drop(["homepage"], axis=1, inplace=True)

In [ ]:
movies.loc[movies.release_date.isna()]

In [ ]:
credits.isna().sum()

In [ ]:
credits.loc[4553]

In [ ]:
movies.drop(4553, inplace=True)
credits.drop(4553, inplace=True)
credits.reset_index(drop=True, inplace=True)
movies.reset_index(drop=True, inplace=True)

In [ ]:
movies.isna().sum()

In [ ]:
movies.loc[movies.overview.isna()]

In [ ]:
movies.genres[0]

**genres and ither columns have json listed values**

In [ ]:
j = json.loads(movies.genres[0])
j

In [ ]:
" ".join("".join(gen["name"].split()) for gen in j)

this above line explained:
we cycle throught the list j and we split for the reason of getting genres like "Science Fiction" which have a space in between into a single word <br>
afterwards we join all the genres into a string, where they are separated by space

**We mostly need the genres and keywords columns** <br>
so we will be mostly using those now and put them in the TfIdfVectorizer

In [ ]:
def genres_and_keywords_to_string(row):
  genres = json.loads(row["genres"])
  genres = " ".join("".join(gen["name"].split()) for gen in genres)

  keywords = json.loads(row["keywords"])
  keywords = " ".join("".join(key["name"].split()) for key in keywords)
  return "%s %s" % (genres, keywords)

In [ ]:
# make new column for the string we manufacture
movies["string"] = movies.apply(genres_and_keywords_to_string, axis=1)

In [ ]:
tfidf = TfidfVectorizer(max_features=2000)

In [ ]:
X = tfidf.fit_transform(movies["string"])

In [ ]:
X

experimetn with max_features and see which works best

In [ ]:
movieindex = pd.Series(movies.index, index=movies.title)

In [ ]:
movieindex

In [ ]:
import re
# use this to search for some harry potter movie
# also check the cut off date on this dataset

In [ ]:
idx = movieindex["The Dark Knight"]
idx

In [ ]:
query = X[idx]
query

In [ ]:
query.toarray()

In [ ]:
scores = cosine_similarity(query, X)
scores

this is a 2d array with shape 1xN and now we will make it 1d using flatten

In [ ]:
scores = scores.flatten()

In [ ]:
plt.plot(scores)

we want the scores to go in descending order and we do not care about the actual score, we just need the ranking of the movies, so we will use (-scores)<br><br>
also, we do not need the sorted values. we need the index (argument) of the sorted values, so we sill use argsort

In [ ]:
(-scores).argsort()

In [ ]:
plt.plot(scores[(-scores).argsort()])

In [ ]:
recommended_idx = (-scores).argsort()[1:11]

In [ ]:
movies["title"].iloc[recommended_idx]

In [ ]:
movies["title"].loc[recommended_idx]

In [ ]:
# make a function for all
def recommend(title):
    ind = movieindex[title]
    if type(ind) == pd.Series:
        ind = ind.iloc[0]

    query = X[ind]
    scores = cosine_similarity(query, X)

    scores = scores.flatten()

    recommended_ind = (-scores).argsort()[1:11]

    return movies.title.loc[recommended_ind]

In [ ]:
print(recommend("Mortal Kombat"))

In [ ]:
for t in movies.title:
    if t.startswith("Harry Potter"):
        print(t)

In [ ]:
for t in movies.title:
    if t.startswith("Fast"):
        print(t)

In [ ]:
print(recommend("Harry Potter and the Chamber of Secrets"))